<a href="https://colab.research.google.com/github/jamiepratt/lingq/blob/main/spacy_bulk_parse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Access Google Drive Directory and Google Sheets

Mount your Google Drive to access files from it.

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Specify the directory in your Google Drive that you want to access.

In [15]:
# Replace 'your-directory-path' with your specific directory path
sub_dir = "lingq_texts/pl/Daily Polish Story/"  #@param {type:"string"}
drive_directory = '/content/drive/MyDrive/' + sub_dir
drive_directory

'/content/drive/MyDrive/lingq_texts/pl/Daily Polish Story/'

In [16]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [17]:
import spacy

#@title Choose a language model
model = "pl_core_news_lg" #@param ["pl_core_news_lg", "ca_core_news_sm", "da_core_news_sm", "de_core_news_sm", "el_core_news_sm", "en_core_web_sm", "es_core_news_sm", "fi_core_news_sm", "fr_core_news_sm", "hr_core_news_sm", "it_core_news_sm", "ja_core_news_sm", "ko_core_news_sm", "lt_core_news_sm", "mk_core_news_sm", "nb_core_news_sm", "nl_core_news_sm", "pt_core_news_sm", "ro_core_news_sm", "sl_core_news_sm", "sv_core_news_sm", "ru_core_news_sm", "uk_core_news_sm", "xx_ent_wiki_sm", "xx_sent_ud_sm", "zh_core_web_sm"]
!python -m spacy download {model}

spacy.prefer_gpu()

nlp = spacy.load(model)

2024-01-20 11:04:50.214548: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 11:04:50.214642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-20 11:04:50.217207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-20 11:04:51.699064: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.7/573.7 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_lg')


### Step 3: Iterate Through Files and Output Content of .txt Files

Define a function to iterate through files in a directory and print the content of `.txt` files.

In [18]:
# doc = nlp("Świetnie! Tak Kasia opowiadała o tym, że była głodna i zrobiła sobie kanapkę. Teraz czas na pytania i odpowiedzi. Jak zawsze zachęcam do głośnego odpowiadania na pytania. Ale jeśli wolisz, możesz tylko posłuchać jak ja na nie odpowiadam. Zaczynajmy! Kasia była głodna. Czy Kasi chciało się jeść? Tak, ona była głodna, czyli chciało jej się jeść. Czy Kasia była najedzona? Nie, ona nie była najedzona. Ona była głodna. Kto był głodny? Kasia. Ona była głodna. Kasia poszła do kuchni i otworzyła lodówkę. Gdzie poszła Kasia? Ona poszła do kuchni. Czy Kasia otworzyła okno w kuchni? Nie, ona nie otworzyła okna. Co otworzyła Kasia? Lodówkę. Ona otworzyła lodówkę. Czy Kasia otworzyła czy zamknęła lodówkę? Otworzyła. Ona otworzyła lodówkę. Ona nie zamknęła lodówki. W lodówce był ser, kiełbasa i masło. Czy w lodówce były owoce? Nie, w lodówce nie było owoców. Co było w lodówce? Ser, kiełbasa i masło. W lodówce był ser, kiełbasa i masło. Czy ser był na stole czy w lodówce? W lodówce. Ser był w lodówce. Kasia nie miała ochoty na kiełbasę. Czy Kasia chciała zjeść kiełbasę? Nie, ona nie chciała kiełbasy. Ona nie miała ochoty na kiełbasę. Na co nie miała ochoty Kasia? Na kiełbasę. Ona nie miała ochoty na kiełbasę. Kto nie miał ochoty na kiełbasę? Kasia. Ona nie miała ochoty na kiełbasę. Ona wzięła ser i masło. Co wzięła z lodówki Kasia? Ser i masło. Ona wzięła ser i masło. Czy Kasia wzięła kiełbasę? Nie, ona nie wzięła kiełbasy. Ona wzięła ser i masło. Kto wziął ser i masło? Kasia. Kasia wzięła ser i masło. Kasia otworzyła szafkę. Czy Kasia zamknęła szafkę? Nie, ona nie zamknęła szafki. Ona otworzyła szafkę. Czy Kasia otworzyła okno? Nie, nie okno. Ona otworzyła szafkę. Co otworzyła Kasia? Szafkę. Ona otworzyła szafkę. W szafce był chleb. Czy szafka była pusta? Nie, szafka nie była pusta. W szafce był chleb. Czy w szafce była kiełbasa? Nie, w szafce nie było kiełbasy. Co było w szafce? Chleb. W szafce był chleb. Kasia wzięła nóż i ukroiła dwie kromki chleba. Czy Kasia ukroiła dwa kawałki chleba? Tak, ona ukroiła dwie kromki chleba, czyli dwa kawałki chleba. Czym Kasia ukroiła chleb? Nożem. Ona wzięła nóż i ukroiła chleb. Ile kromek chleba ukroiła Kasia? Dwie czy trzy? Dwie. Ona ukroiła dwie kromki chleba. Czy Kasia ukroiła cztery kromki chleba? Nie, ona nie ukroiła czterech kromek chleba. Ona ukroiła tylko dwie kromki chleba. Potem ukroiła plasterek sera. Czy Kasia ukroiła cienki kawałek sera? Tak, ona ukroiła plasterek sera, czyli cienki kawałek sera. Plasterek to cienki kawałek czegoś. Może być plasterek sera, plasterek kiełbasy, plasterek szynki. Czy Kasia najpierw ukroiła dwie kromki chleba, a potem plasterek sera? Tak, ona ukroiła najpierw chleb, a potem plasterek sera. Co ukroiła Kasia? Ser. Ona ukroiła plasterek sera. Kasia posmarowała chleb masłem. Czym Kasia posmarowała chleb? Masłem. Ona posmarowała chleb masłem. Co Kasia posmarowała masłem? Chleb. Ona posmarowała chleb masłem. Co zrobiła Kasia? Ona posmarowała chleb masłem. Kasia włożyła ser między posmarowane kawałki chleba. Gdzie Kasia włożyła ser? Między chleb. Ona włożyła ser między posmarowane kawałki chleba. Czy Kasia włożyła kiełbasę między kawałki chleba? Nie, ona nie włożyła kiełbasy. Co Kasia włożyła między kawałki chleba? Ser. Ona włożyła ser między kawałki chleba. Co zrobiła Kasia? Ona włożyła ser między posmarowane kawałki chleba. Ona położyła kanapkę na talerzu. Czy Kasia położyła kanapkę na stole? Nie, ona nie położyła kanapki na stole. Gdzie Kasia położyła kanapkę? Na talerzu. Ona położyła kanapkę na talerzu. Czy Kasia położyła ciasto na talerzu? Nie, ona nie położyła ciasta na talerzu. Co Kasia położyła na talerzu? Kanapkę. Ona położyła kanapkę na talerzu. Kasia usiadła przy stole i zjadła kanapkę. Czy Kasia usiadła obok stołu? Tak, ona usiadła przy stole, czyli obok stołu. Czy Kasia zjadła ciasto? Nie, ona nie zjadła ciasta. Co zjadła Kasia? Kanapkę. Ona zjadła kanapkę. Co zrobiła Kasia? Ona usiadła przy stole i zjadła kanapkę. Doskonale. W ten sposób kończy się dzisiejsza historyjka. Kończą się również pytania i odpowiedzi. Posłuchaj całej lekcji jeszcze kilka razy, a potem zapraszam na dalszy ciąg opowiadania o Kasi. Jeśli chcesz dowiedzieć się, co wydarzyło się potem, posłuchaj następnej historyjki po polsku. Pa, pa!")
# for s in doc.sents:
#   if len(s) < 4:
#     print(s.text)

In [21]:
# see https://github.com/jamiepratt/lingq/blob/main/spacy_experiments.ipynb for how I calculated this:
data_cols = ["filename", "directory", "token_no", "term", "lemma", "pos", "part of speech", "sentence", "morph", "tags"]

pos_to_skip = ["PUNCT", "CCONJ", "SPACE", "X", "INTJ", "SYM"]

(data_cols[3], data_cols[7])

('term', 'sentence')

In [20]:
import os

def spacy_data_from_txt_files_content(directory):
    encountered_words = {}
    data = []
    for root, dirs, files in os.walk(directory):
        for file_name in files:
            if file_name.endswith('.txt'):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'r') as file:
                    print(f"Processing contents of {file_path}:\n")
                    doc = nlp(file.read())
                    for s in doc.sents:
                      if len(s) > 4:
                        for t in s:
                          if t.pos_ not in pos_to_skip:
                            word_sentence = (t.text, t.sent.text)
                            if word_sentence not in encountered_words:
                              encountered_words[word_sentence] = True
                              explained = spacy.explain(t.pos_)
                              t_data = [file_name, root, t.i, t.text, t.lemma_,
                                        t.pos_, explained, t.sent.text, f'{t.morph}',
                                        " ".join([explained] + f'{t.morph}'.split("|"))]
                              data.append(t_data)

    return data

SyntaxError: expected ':' (<ipython-input-20-7954072025e6>, line 13)

In [ ]:
spacy_data = spacy_data_from_txt_files_content(drive_directory)

NameError: name 'spacy_data' is not defined

In [ ]:

# Create a New Google Sheet
sh = gc.create(f'spaCy analysis of {drive_directory}')
# Open the Google Sheet with gspread
worksheet = sh.get_worksheet(0)  # '0' refers to the first sheet

# Update the sheet with DataFrame values
worksheet.update('A1', [data_cols] + spacy_data)

{'spreadsheetId': '1V9j5NIuw53wVeKWQAL_Zd_onCKhiaDT932o5NyvX8ss',
 'updatedRange': 'Sheet1!A1:J18783',
 'updatedRows': 18783,
 'updatedColumns': 10,
 'updatedCells': 187830}